# Haralick descriptors

My Haralick descriptors implementation, the tests suggest that it's correct.
Now I have to plug it on my inpaint algorithm to be able to test it.

In [8]:
import numpy as np
#import imageio as imgio

# function to compute the co-ocurrence matrices of a neighbourhood
def getCoocurrenceMatrices(im, gray_lvs):
   mat = np.zeros((4, gray_lvs, gray_lvs), dtype=float)
   grays = range(gray_lvs)
   #horizontal co-ocurrence
   # removes the last columm of the predecessor image
   im_p_h = im[:, :-1]
   # vertical co-ocurrence
   # removes the last row of the predecessor image
   im_p_v = im[:-1]
   # principal diagonnal co-ocurrence
   # removes last row and columm
   im_p_pd = im_p_h[:-1]
   # second diagonal co-ocurrence
   # removes last row and first columm
   im_p_sd = im[:-1, 1:]
   # computes for each gray level
   for gray in grays:
      # finds the indexes of all precedent pixels that are of this gray level
      indexes_h = np.where(im_p_h == gray)
      indexes_v = np.where(im_p_v == gray)
      indexes_pd = np.where(im_p_pd == gray)
      indexes_sd = np.where(im_p_sd == gray)
      # gets the pixels that are their successors
      succe_h = im[indexes_h[0], indexes_h[1] + 1]
      succe_v = im[indexes_v[0] + 1, indexes_v[1]]
      succe_pd = im[indexes_pd[0] + 1, indexes_pd[1] + 1]
      succe_sd = im[indexes_sd[0] + 1, indexes_sd[1]]
      # computes the co-ocurrences
      for j in range(succe_h.shape[0]):
         mat[0, gray, int(succe_h[j])] += 1
      for j in range(succe_v.shape[0]):
         mat[1, gray, int(succe_v[j])] += 1
      for j in range(succe_pd.shape[0]):
         mat[2, gray, int(succe_pd[j])] += 1
      for j in range(succe_sd.shape[0]):
         mat[3, gray, int(succe_sd[j])] += 1
   for i in range(4):
      mat[i] /= np.sum(mat[i])
   return mat

def getMeansAndStds(coocMats, gray_lvs):
   grays = np.arange(gray_lvs)
   mr = np.zeros(4, dtype=float)
   mc = np.zeros(4, dtype=float)
   stdr = np.zeros(4, dtype=float)
   stdc = np.zeros(4, dtype=float)
   for i in range(4):
      # gets the sum of the probabilities of the rows
      p_sum_r = np.sum(coocMats[i], axis=1)
      # gets the sum of the probabilities of the columns
      p_sum_c = np.sum(coocMats[i], axis=0)

      mr[i] = np.sum(np.multiply(grays, p_sum_r))
      mc[i] = np.sum(np.multiply(grays, p_sum_c))

      stdr[i] = np.sum(np.multiply(np.power(grays - mr[i], 2), p_sum_r))
      stdc[i] = np.sum(np.multiply(np.power(grays - mc[i], 2), p_sum_c))

   return mr, mc, np.sqrt(stdr), np.sqrt(stdc);

def getMaxProbabilities(coocMats):
   max_ps = np.zeros(4, dtype=float)
   for mat in range(4):
      max_ps = coocMats[mat].max()
   return max_ps

def getCorrelation(coocMats, mr, mc, stdr, stdc, gray_lvs):
   correlations = np.zeros(4, dtype=float)
   grays = np.arange(gray_lvs)
   for mat in range(4):
      if (stdr[mat] != 0) and (stdc[mat] != 0):
         for i in grays:
            for j in grays:
               correlations[mat] += np.divide(np.multiply(np.multiply(i - mr[mat], j - mc[mat]), coocMats[mat][i][j]), stdr[mat] * stdc[mat])
      else: correlations[mat] = 0.0
   return correlations

def getContrast(coocMats, gray_lvs):
   grays = np.arange(gray_lvs)
   contrasts = np.zeros(4, dtype=float)
   for mat in range(4):
      for i in grays:
         for j in grays:
            contrasts[mat] += np.power(i - j, 2) * coocMats[mat][i][j]
   return contrasts

def getEnergy(coocMats):
   energies = np.zeros(4, dtype=float)
   for mat in range(4):
      energies[mat] = np.sum(np.power(coocMats[mat]))
   return energies

def getHomogeneity(coocMats):
   grays = np.arange(gray_lvs)
   homogeneities = np.zeros(4, dtype=float)
   for mat in range(4):
      for i in grays:
         for j in grays:
            homogeneities[mat] += coocMats[mat][i][j] / (1 + abs(i - j))
   return homogeneities

def getEntropy(coocMats):
   entropies = np.zeros(4, dtype=float)
   for mat in range(4):
      entropies[mat] = -np.sum(np.multiply(coocMats[mat], np.log2(coocMats[mat])))
   return entropies

def getHaralickDescriptors(coocMats, mr, mc, stdr, stdc):
   grays = np.arange(coocMats.shape[1])
   desc = np.zeros((4, 6), dtype=float)
   desc[:, 1] = getCorrelation(coocMats, mr, mc, stdr, stdc, coocMats.shape[1]) # computes the correlation
   for mat in range(4):
      desc[mat][0] = coocMats[mat].max()                             # computes the max probability
      desc[mat][3] = np.sum(np.power(coocMats[mat], 2))              # computes the energy
      for i in grays:
         for j in grays:
            desc[mat][2] += np.power(i - j, 2) * coocMats[mat][i][j] # computes the contrast
            desc[mat][4] += coocMats[mat][i][j] / (1 + abs(i - j))   # computes the homogeneity
            if coocMats[mat][i][j] != 0:
               desc[mat][5] += coocMats[mat][i][j] * np.log2(coocMats[mat][i][j])

   desc[:, 5] *= -1.0

   return desc

mat_3 = getCoocurrenceMatrices(np.array([[0, 2, 1], [2, 1, 3], [1, 3, 2]]), 4)

mr_3, mc_3, sr_3, sc_3 = getMeansAndStds(mat_3, 4)

desc_3 = getHaralickDescriptors(mat_3, mr_3, mc_3, sr_3, sc_3)
print("Haralick Descriptors image 3x3\n", desc_3)

im_10 = np.ones((10,10))
im_10[:, ::2] = 0

coocMats_10 = getCoocurrenceMatrices(im_10, 2)
mr_10, mc_10, stdr_10, stdc_10 = getMeansAndStds(coocMats_10, 2)
descs_10 = getHaralickDescriptors(coocMats_10, mr_10, mc_10, stdr_10, stdc_10)
print("\nHaralick Descriptors image 10x10\n", descs_10)

im_100 = np.ones((100,100))
im_100[::2, 0::2] = 0

coocMats_100 = getCoocurrenceMatrices(im_100, 2)
mr_100, mc_100, stdr_100, stdc_100 = getMeansAndStds(coocMats_100, 2)
desc_100 = getHaralickDescriptors(coocMats_100, mr_100, mc_100, stdr_100, stdc_100)
print("\nHaralick Descriptors image 100x100\n", desc_100)

Haralick Descriptors image 3x3
 [[ 0.33333333 -0.42640143  2.5         0.27777778  0.41666667  1.91829583]
 [ 0.33333333 -0.42640143  2.5         0.27777778  0.41666667  1.91829583]
 [ 0.5         1.          1.          0.375       0.5         1.5       ]
 [ 0.5         1.          0.          0.375       1.          1.5       ]]

Haralick Descriptors image 10x10
 [[ 0.55555556 -1.          1.          0.50617284  0.5         0.99107606]
 [ 0.5         1.          0.          0.5         1.          1.        ]
 [ 0.55555556 -1.          1.          0.50617284  0.5         0.99107606]
 [ 0.55555556 -1.          1.          0.50617284  0.5         0.99107606]]

Haralick Descriptors image 100x100
 [[ 0.5        -0.33331822  0.5         0.37501275  0.75        1.4999632 ]
 [ 0.5        -0.33331822  0.5         0.37501275  0.75        1.4999632 ]
 [ 0.49994898 -0.33331822  0.50005102  0.37502551  0.74997449  1.49990381]
 [ 0.50005102 -0.33328799  0.49994898  0.37502551  0.75002551  1.4999